IMAGE LOADING

In [1]:
from transformers import BlipProcessor, BlipForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset
from PIL import Image
import pandas as pd
import re
import io

/Users/qrivnel/Desktop/GraduationThesis/TrueColorsBackend-version-1/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
splits = {'train': 'data/combined_train.parquet', 
          'validation': 'data/validation-00000-of-00001-f5d78556e52cceb7.parquet'}
dataset = pd.read_parquet(splits["train"])

PREPROCESSİNG

In [3]:
reviews_sample = dataset.sample(6000, random_state=42)

reviews_sample = reviews_sample.drop(columns=["imgid", "captions"])
reviews_sample['image'] = reviews_sample['image'].apply(lambda x: x['bytes'])
reviews_sample = reviews_sample.rename(columns={"image": "image_bytes", "detailed_caption": "captions"})


colors = [
    "Kırmızı",
    "Yeşil",
    "Mavi",
    "Sarı",
    "Turuncu",
    "Pembe",
    "Mor",
    "Kahverengi",
    "Siyah",
    "Beyaz",
    "Gri",
    "Lacivert",
    "Altın",
    "Gümüş",
    "Zeytin",
    "Fuşya",
    "Bordo",
    "Mercan",
    "Ceviz",
    "Zümrüt",
    "Teal",
    "Bej",
    "kırmızı",
    "yeşil",
    "mavi",
    "sarı",
    "turuncu",
    "pembe",
    "mor",
    "kahverengi",
    "siyah",
    "beyaz",
    "gri",
    "lacivert",
    "altın",
    "gümüş",
    "zeytin",
    "fuşya",
    "bordo",
    "mercan",
    "ceviz",
    "zümrüt",
    "teal",
    "bej"
];

def preprocess_captions(caption):
    sentences = []
    caption = re.sub("\s+", " ", caption).strip()
    for color in colors:
      new_sentences = re.findall(rf'[^.!?]*\b{color}\b[^.!?]*[.!?]', caption, re.IGNORECASE)
      for new_sentence in new_sentences:
         if new_sentence not in sentences:
            sentences.insert(0, new_sentence)
    new_caption = ""
    for sentence in sentences:
        new_caption += sentence
    return new_caption

reviews_sample["captions"] = reviews_sample["captions"].apply(preprocess_captions)

TRAIN SPLIT

In [4]:
hf_dataset = Dataset.from_pandas(reviews_sample)

train_test_split = hf_dataset.train_test_split(train_size=0.9, seed=42)
train_dataset = train_test_split["train"]
val_dataset = train_test_split["test"]

In [5]:
model_name = "Salesforce/blip-image-captioning-base"
model = BlipForConditionalGeneration.from_pretrained(model_name)
processor = BlipProcessor.from_pretrained(model_name)

def preprocess_data(example):
    image_bytes = example["image_bytes"]
    caption = example["captions"]

    images = [Image.open(io.BytesIO(img_bytes)) for img_bytes in image_bytes]

    inputs = processor(
        images=images,
        text=caption,
        padding=True,
        max_length=39,
        truncation=True,
        return_tensors="pt",
    )
    inputs['labels'] = inputs['input_ids'].clone()

    return inputs

train_dataset = train_dataset.map(preprocess_data, batched=True)
val_dataset = val_dataset.map(preprocess_data, batched=True)

Map: 100%|██████████| 600/600 [00:07<00:00, 76.04 examples/s]


In [6]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./trainer_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
    metric_for_best_model="eval_loss",
    fp16=False,
)

# Trainer oluşturma
trainer = Seq2SeqTrainer(
    model=model,  # Model
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor,
)

trainer.train()

trainer.save_model("./trained_model")


/Users/qrivnel/Desktop/GraduationThesis/TrueColorsBackend-version-1/myenv/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/sb/2_gg7fmj5l35q4cqx1x15wv40000gn/T/ipykernel_20113/2572126915.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
  0%|          | 100/27000 [01:19<5:49:25,  1.28it/s]

{'loss': 6.9861, 'grad_norm': 16.782052993774414, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}


  1%|          | 200/27000 [02:37<5:48:06,  1.28it/s]

{'loss': 5.0176, 'grad_norm': 20.200523376464844, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.04}


  1%|          | 300/27000 [03:55<5:53:57,  1.26it/s]

...

{'loss': 0.4788, 'grad_norm': 13.052371978759766, 'learning_rate': 0.0, 'epoch': 5.0}


                                                       
100%|██████████| 27000/27000 [7:09:08<00:00,  1.20it/s]

{'eval_loss': 1.4753512144088745, 'eval_runtime': 152.747, 'eval_samples_per_second': 3.928, 'eval_steps_per_second': 3.928, 'epoch': 5.0}


100%|██████████| 27000/27000 [7:09:13<00:00,  1.05it/s]


{'train_runtime': 25753.337, 'train_samples_per_second': 1.048, 'train_steps_per_second': 1.048, 'train_loss': 1.2166555875142415, 'epoch': 5.0}
